# Sixth Stage

### Investigate into the impact of flash orders on the simulation results

In [1]:
from optimization import *
from flash_order import *

In [12]:
target_stock_code = '2330'
side = 'bid'
ts = 20
tm = 10
max_dur = 200
trick_thres = 100

In [13]:
# First get simulation results
sim_res = read_pkl_helper(os.path.join(PROJECT_DIR, f'optres/stock_code={target_stock_code}/side={side}/ts={ts}/tm={tm}', 'full_res.pkl'))

In [14]:
# Get flash orders
foc = FlashOrderCalculator(target_stock_code)
foc.classify(max_dur_ms=max_dur, trick_trade_thres_ms=trick_thres)

2022-12-06 21:51:31.430 | INFO     | get_data.py         :   36 | Got cache at /Users/anthonywah/Projects/mafs6100-order-placement-strategy/cache/2330.pkl - 12.53s
ClassifyingFlashOrder: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [00:05<00:00, 31.49it/s]


In [15]:
len(sim_res)

6878

In [16]:
# For each simulation round, check if there is a flash order during the order placement period

for col in ['fo', 'fo_bid', 'fo_ask']:  # Record matched fo in index lists
    if col in sim_res.columns:
        sim_res = sim_res.drop(col, axis=1).reset_index(drop=True)
    sim_res.loc[:, col] = [[] for i in range(len(sim_res))]

one_fod = foc.fod[max_dur][trick_thres]
for fo in tqdm.tqdm(one_fod, desc="FindingFlashOrders", ncols=200, total=len(one_fod)):
    matched = sim_res.loc[(sim_res['start_ts'] <= fo['start_ms']) & (sim_res['fill_ts'] >= fo['end_ms'])].index
    if not len(matched):
        continue
    
    # Store case indexes
    for i in matched:
        sim_res.at[i, 'fo'].append(fo['case_index'])
    
    # Side breakdown
    if fo['side'] == 'bid':
        sim_res.at[i, 'fo_bid'].append(fo['case_index'])
    else: 
        sim_res.at[i, 'fo_ask'].append(fo['case_index'])

FindingFlashOrders: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2356/2356 [00:01<00:00, 1467.87it/s]


In [17]:
# Get simulation rounds with flash orders
sim_res_dict = {
    'Normal'.ljust(15): sim_res.copy(), 
    'With FO'.ljust(15): sim_res.loc[sim_res['fo'].apply(len) > 0, :].reset_index(drop=True), 
    'No FO'.ljust(15): sim_res.loc[sim_res['fo'].apply(len) == 0, :].reset_index(drop=True), 
    'With bid FO'.ljust(15): sim_res.loc[sim_res['fo_bid'].apply(len) > 0, :].reset_index(drop=True), 
    'No bid FO'.ljust(15): sim_res.loc[sim_res['fo_bid'].apply(len) == 0, :].reset_index(drop=True), 
    'With ask FO'.ljust(15): sim_res.loc[sim_res['fo_ask'].apply(len) > 0, :].reset_index(drop=True), 
    'No ask FO'.ljust(15): sim_res.loc[sim_res['fo_ask'].apply(len) == 0, :].reset_index(drop=True), 
}

In [18]:
# Calculation % of different fill types
log_info(f'Cases breakdown: ')
for k, i_sim_res in sim_res_dict.items():
    count_dict = i_sim_res.groupby('case')['date'].count().to_dict()
    msg = f'{k}: Count = {len(i_sim_res):>7}'
    for case in ['INIT', 'EOQREP', 'REP', 'TAKE']:
        msg += f' ; {case} = {count_dict.get(case, 0.) * 100 / len(i_sim_res):>6.2f}%'
    msg += f' ; Avg PnL = {i_sim_res["pnl"].mean():>6.3f} ; Stdev = {i_sim_res["pnl"].std():>6.3f}'
    log_info(msg)

2022-12-06 21:51:39.764 | INFO     | 652576031.py        :    2 | Cases breakdown: 
2022-12-06 21:51:39.774 | INFO     | 652576031.py        :    9 | Normal         : Count =    6878 ; INIT =   2.14% ; EOQREP =   6.67% ; REP =  15.05% ; TAKE =  76.14% ; Avg PnL = -5.888 ; Stdev =  8.557
2022-12-06 21:51:39.777 | INFO     | 652576031.py        :    9 | With FO        : Count =     730 ; INIT =   2.33% ; EOQREP =   9.59% ; REP =  18.49% ; TAKE =  69.59% ; Avg PnL = -5.507 ; Stdev =  8.792
2022-12-06 21:51:39.781 | INFO     | 652576031.py        :    9 | No FO          : Count =    6148 ; INIT =   2.11% ; EOQREP =   6.33% ; REP =  14.64% ; TAKE =  76.92% ; Avg PnL = -5.933 ; Stdev =  8.528
2022-12-06 21:51:39.783 | INFO     | 652576031.py        :    9 | With bid FO    : Count =     321 ; INIT =   1.87% ; EOQREP =  10.59% ; REP =  16.20% ; TAKE =  71.34% ; Avg PnL = -6.550 ; Stdev =  9.072
2022-12-06 21:51:39.787 | INFO     | 652576031.py        :    9 | No bid FO      : Count =    6557 ;

---